# Import well tops data

In [1]:
import numpy as np
import pandas as pd
import os
import glob
import lasio
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

/Users/Heizenberg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [455]:
# Reading wireline logs 
dir_='../Wells/'
list_ = []
for filename in os.listdir(dir_):
    if "Additional" not in filename:
        g = glob.glob(dir_+filename+'/Wireline Data-Merged/*Final*')
        las = lasio.read(g[0])
        df = las.df()
        #df_ = pd.read_excel(file[0],'Formation Summary',skiprows=1)
        df['Wellname']=filename.lower().strip()
        df['MD'] = df.index
        list_.append(df)
    else:
        for f in os.listdir(dir_+filename+'/Final_las/'):
            las = lasio.read(dir_+filename+'/Final_las/'+f)
            df = las.df()
            name = os.path.splitext(f)[0]
            df['Wellname']= name[:-6].lower().strip()
            df['MD'] = df.index
            list_.append(df)
        
big_frame = pd.concat(list_, axis = 0, ignore_index = True)

/Users/Heizenberg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



# Clean Wireline Data

In [456]:
big_frame['Wellname'] = big_frame['Wellname'].str.replace(' ','-')
big_frame['Wellname']=big_frame['Wellname'].str.replace('a-','a')
big_frame['Wellname'] = big_frame['Wellname'].str.replace('c-','c')
big_frame['Wellname'] = big_frame['Wellname'].str.replace('b-5','b5')

In [457]:
DirtyWell = big_frame[big_frame['Wellname']=='cheal-a6_st1'].index
DirtyWell
big_frame.drop(DirtyWell,inplace=True,axis=0)

# Read Deviation data

In [458]:
# Read Divtation survey
dir_ = '../Deviation/'
list_ = []
for filename in os.listdir(dir_):
    mydf = pd.read_csv(dir_+filename,sep='\t')
    mydf['Wellname']=filename[:-4].lower().strip()
    list_.append(mydf)
dev_data = pd.concat(list_, axis = 0, ignore_index = True)

In [459]:
dev_data = dev_data.rename(columns={'MD(m)':'MD'})

In [460]:
dev_data = dev_data.rename(columns={'X(m)':'X','Y(m)':'Y','Z(m)':'TVDSS'})

### Explorate deviation and wireline

In [ ]:
big_frame.head()

In [8]:
from sklearn import preprocessing


### Interpolation

In [12]:
dev_data[dev_data['TVD(m)'].isnull()]

,X,Y,TVDSS,TVD(m),MD,Wellname


# Merge Wireline and Deviation

In [461]:
big_frame = big_frame.sort_values(['Wellname','MD'])

In [462]:
dev_data = dev_data.sort_values(['Wellname','MD'])

In [463]:
big_frame['MD']= round(big_frame['MD'] * 3.28084)

In [464]:
dev_data['MD'] = round(dev_data['MD']*3.28084)

In [465]:
df = big_frame.merge(dev_data,on=['Wellname','MD'],how='inner')

In [ ]:
df.shape

In [466]:
df['GR'] = df['GR'].fillna(df['GR_CORR'])
df['NEUT'] = df['NEUT'].fillna(df['NEUT_CORR'])
df['RESD'] = df['RESD'].fillna(df['RESD_CORR'])

In [467]:
 df.drop(['BS','CALI', 'DENS', 'DENS_CORR', 'DRHO', 'DTS', 'GR_CORR', 'NEUT_CORR', 'PEF', 'RESD_CORR', 'RESM', 'RESS',
       'RESS_CORR', 'SP', 'TEMP', 'TENS'],axis=1,inplace=True)

# Normalization

In [16]:
from sklearn import preprocessing


In [468]:
x = df[['GR','DTC','NEUT']].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_ = pd.DataFrame(x_scaled)

In [469]:
df[['GR','DTC','NEUT']] = df_

In [470]:
df.dropna(subset=['GR','DTC','NEUT','RESD'],how='all',inplace=True)

In [471]:
df[['GR','DTC','NEUT','RESD']]=df[['GR','DTC','NEUT','RESD']].fillna(-999.25)

In [ ]:
df[df['Y'].isnull()]

In [475]:
# Reading Well-formation
dir_='../'
f = 'Merged_Well_Tops.xlsx'
df_ = pd.read_excel(dir_+f,names=['Wellname','Formation','MD'])
df_['Wellname'] = df_['Wellname'].str.lower().str.strip()
df_['Formation'] = df_['Formation'].str.lower().str.strip()

In [ ]:
col = df.columns.drop(['GR','NEUT','RESD','DTC','TVDSS'])

In [ ]:
df.head()

Fill missing values in X and Y

In [472]:
df[['X','Y']]=df[['X','Y']].fillna(method='bfill')

In [473]:
df[['X','Y']]=df[['X','Y']].interpolate(method='linear',limit_direction='forward',limit=1000)

In [ ]:
df.tail()

### Fill TVD

# Clean Well tops Data

In [476]:
DirtyWell = df_[df_['Wellname']=='cheal b-2'].index
df_.drop(DirtyWell,inplace=True)

In [477]:
DirtyWell = df_[df_['Wellname']=='cheal b-3'].index
df_.drop(DirtyWell,inplace=True)

In [26]:
for c in df_['Formation'].unique():
    if(df_[df_['Formation']==c].count()[0]<=16):
        df_=df_[df_['Formation']!=c]

In [478]:
df_['Formation']=df_['Formation'].astype('category').cat.codes

In [479]:
data = devwire.merge(df_,on=['MD','Wellname'],how='outer')

In [ ]:
data.shape

In [480]:
data = data.sort_values(['Wellname','MD'])

In [481]:
data['Formation']=data['Formation'].fillna(method='ffill')

In [482]:
data['Formation']=data['Formation'].fillna(-1)

In [483]:
data['TVD(m)'] = round(data['TVD(m)']*3.28084)

In [484]:
data['TVDSS'] = round(data['TVDSS']*3.28084)

In [487]:
data[data['GR'].isnull()]

,DTC,GR,MD,NEUT,RESD,Wellname,X,Y,TVDSS,TVD(m),Formation


In [486]:
data[['GR','DTC','NEUT','RESD']]=data[['GR','DTC','NEUT','RESD']].interpolate(limit_direction='both')

# Assume X and Y

In [488]:
data[['X','Y']] = data[['X','Y']].interpolate(limit=1000,limit_direction='forward')

In [489]:
data[['X','Y']]=data[['X','Y']].fillna(method='bfill')

# Work on TVD

In [43]:
df1=data

In [490]:
data['TVD(m)'] = data['TVD(m)'].interpolate(limit=1,limit_direction='forward')

In [491]:
data['TVD(m)']=data['TVD(m)'].fillna(data['MD'])

# Interpolate TVDSS

In [ ]:
data = df2

In [ ]:
data['TVDSS']=data['TVDSS'].interpolate(method='polynomial',limit=1,limit_direction='both',order=2)

In [492]:
data.drop(columns=['TVDSS'],inplace=True)

In [ ]:
data.head()

### Clean formations

In [493]:
for c in df_['Formation'].unique():
    if(df_[df_['Formation']==c].count()[0]<=16):
        data[data['Formation']==c] = -1

In [494]:
data['Formation'].value_counts()

-1.0     5219
 5.0      416
 11.0     237
 13.0     235
 3.0      220
 10.0     120
Name: Formation, dtype: int64

# Prepare Model

In [55]:
df = data

In [495]:
grouped = data.groupby('Wellname')

In [496]:
split_well = pd.read_csv('split_wells.csv')

In [497]:
split_well=split_well.fillna(0)

In [498]:
train_=[]
test_ = []
for w in split_well['Train']:
    train_.append(grouped.get_group(w))
for w in split_well['Test']:
    if (w!=0):
        test_.append(grouped.get_group(w))
train = pd.concat(train_,axis=0,ignore_index=True)
test = pd.concat(test_,axis=0,ignore_index=True)

### Classification Problem

In [499]:
X_train = train.drop(['Wellname','Formation'],axis=1)
y_train = train['Formation']
X_test = test.drop(['Wellname','Formation'],axis=1)
y_test = test['Formation']

In [52]:
df[df['NEUT'].isnull()]

,DTC,GR,MD,NEUT,RESD,Wellname,X,Y,TVDSS,TVD(m)


# Base Model

In [53]:
from sklearn.linear_model import LogisticRegression

In [500]:
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X_train, y_train)

In [501]:
print(clf.score(X_test,y_test))

0.5197132616487455


In [56]:
from sklearn.tree import DecisionTreeClassifier

In [284]:
X_test.head()

,DTC,GR,MD,NEUT,RESD,X,Y,TVD(m),pred
0,0.485168,0.546043,949.0,0.443780,25.1830,2622382.74,6201193.400,949.0,1
1,0.482754,0.531214,949.0,0.447368,24.6710,2622382.74,6201193.400,949.0,1
2,0.472147,0.500213,976.0,0.415072,25.8755,2622382.68,6201193.325,971.0,1
3,0.461541,0.469211,993.0,0.382775,27.0800,2622382.62,6201193.250,993.0,1
4,0.457212,0.477978,993.0,0.381579,28.3880,2622382.62,6201193.250,993.0,1


In [285]:
X_train.head()

,DTC,GR,MD,NEUT,RESD,X,Y,TVD(m)
0,-499.496805,0.379638,121.3,-499.432416,-497.595,2622409.685,6201122.03,2622.0
1,-999.250000,0.072334,274.0,-999.250000,-999.250,2622372.540,6201194.73,274.0
2,-999.250000,0.062044,274.0,-999.250000,-999.250,2622372.540,6201194.73,274.0
3,-999.250000,0.124151,310.0,-999.250000,-999.250,2622372.390,6201194.57,310.0
4,-999.250000,0.126444,310.0,-999.250000,-999.250,2622372.390,6201194.57,310.0


In [508]:
clf = DecisionTreeClassifier().fit(X_train,y_train)
print(clf.score(X_test,y_test))

0.6415770609318996


In [309]:
from sklearn.ensemble import RandomForestClassifier

In [503]:
clf = RandomForestClassifier().fit(X_train,y_train)
print(clf.score(X_test,y_test))

0.7670250896057348


/Users/Heizenberg/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [521]:
y_pred = clf.predict(X_test)

In [522]:
o = test
o['Formation'] = y_pred
pred = o.groupby(['Wellname','Formation'])['MD'].min()

In [523]:
comparision = df_.merge(pred,on=['Wellname','Formation'],how='inner')

In [525]:
np.mean((comparision['MD_x']-comparision['MD_y'])/comparision['MD_x'])

0.3618206401399975

In [431]:
def mycustom(clf,X,y):
    y_pred = clf.predict(X)
    o1 = train.loc[X.index]
    o1['Formation'] = y_pred
    pred = o1.groupby(['Wellname','Formation'])['MD'].min()
    comparision = o1.merge(df_,on=['Wellname','Formation'],how='inner')
    score = np.abs(np.mean((comparision['MD_x']-comparision['MD_y'])/comparision['MD_x']))
    return score

In [436]:
def mycustom1(clf,X,y):
    y_pred = clf.predict(X)
    o1 = test.loc[X.index]
    o1['Formation'] = y_pred
    pred = o1.groupby(['Wellname','Formation'])['MD'].min()
    comparision = o1.merge(df_,on=['Wellname','Formation'],how='inner')
    score = np.abs(np.mean((comparision['MD_x']-comparision['MD_y'])/comparision['MD_x']))
    return score

In [427]:
data.loc[X_train.index].head()

,DTC,GR,MD,NEUT,RESD,Wellname,X,Y,TVD(m),Formation
0,0.389167,0.246102,1401.0,0.315115,56.7357,cheal-1,2622442.48,6201198.67,1401.0,4.0
1,0.386226,0.247740,1401.0,0.314164,52.5651,cheal-1,2622442.48,6201198.67,1401.0,4.0
2,0.366010,0.350292,1854.0,0.407555,28.2807,cheal-1,2622442.27,6201196.88,1854.0,2.0
3,0.366227,0.343867,1854.0,0.425701,28.1603,cheal-1,2622442.27,6201196.88,1854.0,2.0
4,0.346384,0.299118,2326.0,0.427074,14.9702,cheal-1,2622442.61,6201195.46,2326.0,2.0


In [426]:
data.head()

,DTC,GR,MD,NEUT,RESD,Wellname,X,Y,TVD(m),Formation
6292,0.389167,0.246102,142.0,0.315115,56.7357,cheal-1,2622442.48,6201198.67,142.0,3.0
6293,0.389167,0.246102,260.0,0.315115,56.7357,cheal-1,2622442.48,6201198.67,260.0,1.0
6294,0.389167,0.246102,1240.0,0.315115,56.7357,cheal-1,2622442.48,6201198.67,1240.0,4.0
0,0.389167,0.246102,1401.0,0.315115,56.7357,cheal-1,2622442.48,6201198.67,1401.0,4.0
1,0.386226,0.247740,1401.0,0.314164,52.5651,cheal-1,2622442.48,6201198.67,1401.0,4.0


In [432]:
from sklearn.model_selection import GridSearchCV
my_scorer = make_scorer(mycustom, greater_is_better=False)


In [433]:
para ={'criterion': ['gini','entropy'],'min_samples_split' : range(10,500,20),'max_depth': range(1,20,2),'min_samples_leaf':range(1,20,2),'max_features':range(1,8)}
grid_search = GridSearchCV(estimator=DecisionTreeClassifier(),param_grid=para,scoring=mycustom,verbose=1)
grid_search.fit(X_train,y_train)


/Users/Heizenberg/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 35000 candidates, totalling 105000 fits


[Parallel(n_jobs=1)]: Done 105000 out of 105000 | elapsed: 55.0min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['gini', 'entropy'], 'min_samples_split': range(10, 500, 20), 'max_depth': range(1, 20, 2), 'min_samples_leaf': range(1, 20, 2), 'max_features': range(1, 8)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=<function mycustom at 0x1a973a41e0>, verbose=1)

In [513]:
clf = grid_search.best_estimator_
print(grid_search.best_score_)
print(grid_search.best_params_)

0.6203787401333587
{'criterion': 'gini', 'max_depth': 5, 'max_features': 1, 'min_samples_leaf': 5, 'min_samples_split': 450}


In [520]:
clf = DecisionTreeClassifier(max_features=1,min_samples_leaf=5,min_samples_split=450).fit(X_train,y_train)

In [414]:
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin, clone
from sklearn.linear_model import LogisticRegression
from sklearn.utils.estimator_checks import check_estimator
#from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

class CutEstimator(BaseEstimator):

    def __init__(self, base_estimator):
        self.base_estimator = base_estimator

    def fit(self, X, y):
        self._base_estimator = clone(self.base_estimator)
        self._base_estimator.fit(X.drop('Wellname',axis=1),y)
        return self

    def predict(self, X):
        return  self._base_estimator.predict(X.drop('Wellname',axis=1))

# Predict tops

In [59]:
from keras.layers import Dense
from keras.models import Input, Model



Using TensorFlow backend.


In [60]:
from keras.layers import Activation, Lambda


In [61]:
X_train.shape

(4352, 8)

In [222]:
X_train = X_train.values.reshape(X_train.shape[0],1,X_train.shape[1])

In [223]:
X_test = X_test.values.reshape(X_test.shape[0],1,X_test.shape[1])

In [121]:
from keras.utils import np_utils

In [224]:
y_test = np_utils.to_categorical(y_test,num_classes=5)

In [225]:
y_train = np_utils.to_categorical(y_train,num_classes=5)

In [155]:
X_train.shape[0]

4352

In [130]:
y_train=np.argmax(y_train,axis=1)

In [67]:
from tcn import compiled_tcn


In [229]:
model = compiled_tcn(num_feat=8,  # type: int
                 num_classes=5,  # type: int
                 nb_filters=5,  # type: int
                 kernel_size=2,  # type: int
                 max_len=X_train.shape[1],
                 activation='norm_relu',  # type: str
                 nb_stacks=2,
             dilations=[2 ** i for i in range(9)],
                 padding='causal',  # type: str
                 use_skip_connections=True,  # type: bool
                 return_sequences=False,
                 regression=False  # type: bool
                 
                 )

x.shape= (?, 5)
model.x = (?, 1, 8)
model.y = (?, 5)
Adam with norm clipping.


In [234]:
from tcn import TCN

In [238]:
i = Input(batch_shape=(32, 1, 8))

o = TCN(return_sequences=False)(i)  # The TCN layers are here.
o = Dense(5,activation='softmax')(o)

m = Model(inputs=[i], outputs=[o])
m.compile(optimizer='adam', loss='categorical_crossentropy')

m.fit(X_train, y_train, epochs=25, validation_data=(X_test,y_test))

Train on 4352 samples, validate on 2007 samples
Epoch 1/25
4352/4352 [==============================] - 24s 6ms/step - loss: 0.8459 - val_loss: 1.7191
Epoch 2/25
4352/4352 [==============================] - 3s 719us/step - loss: 0.8428 - val_loss: 1.8093
Epoch 3/25
4352/4352 [==============================] - 3s 724us/step - loss: 0.8381 - val_loss: 1.9833
Epoch 4/25
4352/4352 [==============================] - 3s 755us/step - loss: 0.8391 - val_loss: 1.8116
Epoch 5/25
4352/4352 [==============================] - 3s 763us/step - loss: 0.8362 - val_loss: 1.7759
Epoch 6/25
4352/4352 [==============================] - 3s 766us/step - loss: 0.8382 - val_loss: 1.9269
Epoch 7/25
4352/4352 [==============================] - 3s 786us/step - loss: 0.8359 - val_loss: 1.8265
Epoch 8/25
4352/4352 [==============================] - 3s 777us/step - loss: 0.8326 - val_loss: 1.7733
Epoch 9/25
4352/4352 [==============================] - 3s 764us/step - loss: 0.8321 - val_loss: 1.7493
Epoch 10/25
4352/

In [274]:
from sklearn.metrics import r2_score
print(r2_score(y_test,np_utils.to_categorical(y_pred)))

-0.6734483004868406


In [275]:
np_utils.to_categorical(y_pred)

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [276]:
y_test

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [253]:
o = test
o['Formation'] = y_proba
pred = o.groupby(['Wellname','Formation'])['MD'].min()

In [250]:
pred.to_csv('myprediction.csv')

/Users/Heizenberg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [252]:
df_ = df_.sort_values(['Wellname','MD'])
df_.to_csv('actual.csv')

In [254]:
comparision = df_.merge(pred,on=['Wellname','Formation'],how='inner')

In [256]:
comparision['diff']=comparision['MD_x'] - comparision['MD_y']

In [263]:
np.mean(comparision['diff']/comparision['MD_x'])*100

-9.525475370881583

In [230]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Train on 4352 samples, validate on 2007 samples
Epoch 1/10


InvalidArgumentError: Can not squeeze dim[1], expected a dimension of 1, got 5
	 [[{{node metrics_35/accuracy/Squeeze}}]]